In [1]:
import sys
import os
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from src.services.cases import get_single_case, patch_case

from langchain_core.output_parsers import JsonOutputParser

import plotly 
import plotly.graph_objs as go

## Process Description

### Use cases: 

Communication with the Court
1. Casenet: 
- Receipts (Confirmation of Filing)
- Returned (Rejected) Notice
- E-Notice (another party files something on the case on Casenet)

2. TylerTech : IL Cook County
- Only one mail sent 

3. Direct Email from the Court

4. Direct Email to the Court with a response 

Communication with the Prosecutor 
- Acceptance 
- Rejection 
- Request for more information

Communication with the Client
- Initial marketing email


Next Steps: 
- Card should have the following information 
    - Case Number
    - Court Date / Court Time 
    - Court/Court Location
    - Charges 
- Rank the cases by next court date / court time 
- Refresh the open cases every day from Casenet to update the court date / time 
    - A case is closed if we have anything expect disposed 
- Analyze all the emails and update the status and mark them as new when new update based on detemerministic rules
- Level#2: Analyze the text inside the emails 
- Level#3: Analyze the last paid invoices 
- Suggestion from the beginning should contain the appropriate action 

## AI Toolbox Initialisation

In [2]:
OPENAI_API_VERSION = "2024-02-15-preview"
AZURE_OPENAI_KEY = "d7878ff58933461a8fd46e35e018c714"
AZURE_OPENAI_ENDPOINT = "https://fublo.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "gpt-35-turbo-16k"

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"] = AZURE_OPENAI_CHATGPT_DEPLOYMENT

In [3]:
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
)

In [4]:
sys.path.append("..")
os.environ["ROOT_PATH"] = "/Users/aennassiri/Projects/Personal/ticket-washer"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/aennassiri/Projects/Personal/ticket-washer/configuration/fubloo-app-1f213ca274de.json"
)

## Data Collection and Preprocessing

In [5]:
from src.services.emails import GmailConnector


In [6]:
user_id = "ayoub@tickettakedown.com"

In [7]:
gmail_connector = GmailConnector(user_id=user_id)


### Retrieval of all the emails from the inbox

In [10]:
emails = gmail_connector.get_inbox_emails(total_results=10000)

Got the first 20 emails
Got the first 40 emails
Got the first 60 emails
Got the first 80 emails
Got the first 100 emails
Got the first 120 emails
Got the first 140 emails
Got the first 160 emails
Got the first 180 emails
Got the first 200 emails
Got the first 220 emails
Got the first 240 emails
Got the first 260 emails
Got the first 280 emails
Got the first 300 emails
Got the first 320 emails
Got the first 340 emails
Got the first 360 emails
Got the first 380 emails
Got the first 400 emails
Got the first 420 emails
Got the first 440 emails
Got the first 460 emails
Got the first 480 emails
Got the first 500 emails
Got the first 520 emails
Got the first 540 emails
Got the first 560 emails
Got the first 580 emails
Got the first 600 emails
Got the first 620 emails
Got the first 640 emails
Got the first 660 emails
Got the first 680 emails
Got the first 700 emails
Got the first 720 emails
Got the first 740 emails
Got the first 760 emails
Got the first 780 emails
Got the first 800 emails
Got 

In [77]:
print(f"Found total of {len(emails)} emails")
email = emails[34]

Found total of 10000 emails


In [78]:
email

{'id': '18ecf8dd30b42a3c',
 'snippet': 'Your submission EF34388866 was RECALLED BY FILER FOR FILING* by Boone County - Circuit Court on 4/11/24 at 6:37 PM. Below is important information regarding this filing. eFiling Confirmation Number',
 'payload': {'partId': '',
  'mimeType': 'multipart/mixed',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'meyerattorneyservices@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a17:906:2796:b0:a51:aaa7:9f75 with SMTP id j22csp224743ejc;        Thu, 11 Apr 2024 16:46:02 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IHd/bUHZdm8Thd9Ae247MavkaM5mDGjKS4tpd+90+iitu6l9f37XqRObTaDwzrbQnP/bDx9'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:6602:3709:b0:7d6:74d1:e86d with SMTP id bh9-20020a056602370900b007d674d1e86dmr1262068iob.20.1712879161945;        Thu, 11 Apr 2024 16:46:01 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1712879161; cv=none;        d=google.com; s=

In [79]:
gmail_connector.get_timestamp(email).strftime("%Y-%m-%d")

'2024-04-12'

In [80]:
 # Snippet 
gmail_connector.get_snippet(email)


'Your submission EF34388866 was RECALLED BY FILER FOR FILING* by Boone County - Circuit Court on 4/11/24 at 6:37 PM. Below is important information regarding this filing. eFiling Confirmation Number'

In [81]:
# Subject
gmail_connector.get_email_subject(email)   # Subject

'Recalled for Filing - 220660075 - ST V PEYTON N PARROTTE, Boone County - Circuit Court'

In [82]:
# Sender
gmail_connector.render_email_sender(gmail_connector.get_sender(email))

'Missouri Courts eFiling System'

## Email Analysis Functions

### Function to extract the case number from the email

In [83]:
import re

def extract_number_from_string(s):
    output = None
    match = re.search(r'\b\d+\b', s)
    if match:
        output = match.group(0)
    # Format 2316-CR04348
    match = re.search(r'\b\w{4}-\w{2}\w{5}\b', s)
    if match:
        output = match.group(0)

    # Format 24CT-CR00287-01
    match = re.search(r'\b\w{4}-\w{2}\w{5}-\d{2}\b', s)
    if match:
        output = match.group(0)

    if output is not None and len(output) < 5:
        output = None
    return output

In [84]:
import re

# The text strings to search
texts = [
    "Receipt of Submission - 704258650 - ST V GARRETT JACOB GARING, Randolph County - Circuit Court - Huntsville",
    "eNotice - 210900213 - KANSAS CITY V JACKSON FLYNN CONROE, Kansas City Municipal Court",
    "eNotice - 2416-CV09157 - TERENCE TENSLEY V DIRECTOR OF REVENUE, Jackson County - Independence - Civil",
    "eNotice - 2316-CR04348 - ST V SEAN F MOLBY, Jackson County - Independence - Criminal/Traffic",
    "eNotice - 24CT-CR00287-01 - ST V AMANDA MICHELLE RIDDLE, Christian County - Circuit Court",
    "77% OFF is still available!",
]

# Regular expression to extract numbers
pattern = r'\b\d+\b'

# Extracting numbers
extracted_numbers = [extract_number_from_string(text) for text in texts]
extracted_numbers

['704258650',
 '210900213',
 '2416-CV09157',
 '2316-CR04348',
 '24CT-CR00287-01',
 None]

In [85]:
email_template = f"""
Date : {gmail_connector.get_timestamp(email).strftime("%Y-%m-%d")} \n

Subject: {gmail_connector.get_email_subject(email)} \n

Body: {gmail_connector.get_snippet(email)} \n

Sender: {gmail_connector.render_email_sender(gmail_connector.get_sender(email))} \n

"""
email_template

'\nDate : 2024-04-12 \n\n\nSubject: Recalled for Filing - 220660075 - ST V PEYTON N PARROTTE, Boone County - Circuit Court \n\n\nBody: Your submission EF34388866 was RECALLED BY FILER FOR FILING* by Boone County - Circuit Court on 4/11/24 at 6:37 PM. Below is important information regarding this filing. eFiling Confirmation Number \n\n\nSender: Missouri Courts eFiling System \n\n\n'

In [86]:
gmail_connector.get_snippet(email)

'Your submission EF34388866 was RECALLED BY FILER FOR FILING* by Boone County - Circuit Court on 4/11/24 at 6:37 PM. Below is important information regarding this filing. eFiling Confirmation Number'

In [87]:
extract_number_from_string(gmail_connector.get_email_subject(email))

'220660075'

### Cases Status Update Function

In [88]:
case_statuses = {
    "filed": {
        "name": "Case Filed",
        "owner": "Client",
        "mandatory": True,
        "short_description": "Case Filed on Casenet",
        "internal_status": "Case Filed",
        "client_status": "",
        "next_case_status": "",
    },
    "paid": {
        "name": "Case Payment",
        "owner": "Client",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Client Paid",
        "internal_status": "Payment Received",
        "client_status": "",
        "next_case_status": "REVIEW",
        "color": "pending",
    },
    "review": {
        "name": "Case Review and Consultation",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Case Under Review",
        "internal_status": "Under Review",
        "client_status": "",
        "next_case_status": "REJECTED, EOA",
        "color": "orange",
    },
    "rejected": {
        "name": "Case Rejected",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Case Rejected",
        "internal_status": "Case cannot be handled by our attorneys",
        "client_status": "",
        "next_case_status": "REFUND",
        "color": "red",
    },
    "refund": {
        "name": "Case Refund",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "closed",
        "short_description": "Case Refunded",
        "internal_status": "We initiated a full refund. It should be reflected in your account within 5-10 business days. We apologize for any inconvenience.",
        "client_status": "",
        "next_case_status": "CLOSURE",
        "color": "red",
    },
    "sign_engagement_letter": {
        "name": "Sign Engagement Letter",
        "owner": "Client, Lawyer",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Engagement Letter Sign-Off",
        "internal_status": "Case Accepted",
        "client_status": "Our Attorneys have accepted your case",
        "next_case_status": "EOA",
        "color": "orange",
    },
    "warrant_status": {
        "name": "Warrant Status",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Recall Warrant",
        "internal_status": "Warrant",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "eoa": {
        "name": "Entry of Appearance",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "EOA",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT, CLIENT_REQUEST",
        "color": "green",
    },
    "court_response": {
        "name": "Court Response",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Court Response",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "case_update": {
        "name": "Case Update",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Case Update",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "court_returned_for_filing": {
        "name": "Court Returned for Filing",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Court Returned for Filing",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "client_request": {
        "name": "Documentation Request from the Customer",
        "owner": "Client",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Client Intake Form",
        "internal_status": "Our Attorneys have requested more information from you on your case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "rev_int": {
        "name": "Case Documentation & Information Gathering",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Internal Review",
        "internal_status": "Our Attorneys are gathering documentation to back your case",
        "client_status": "",
        "next_case_status": "DEFENSE_REV",
        "color": "orange",
    },
    "defense_dev": {
        "name": "Case Legal Strategy Development",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Review with the Client",
        "internal_status": "Our attorneys are elaborating a strategy to defend your cause",
        "client_status": "",
        "next_case_status": "REC_REQUEST",
        "color": "orange",
    },
    "rec_request": {
        "name": "Case Request for Recommendation",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "RFR Filing",
        "internal_status": "Attorney has initiated negotiations with the Prosecutor",
        "client_status": "",
        "next_case_status": "REC_RECEIVED, REC_DELAYED, REC_REJECT",
    },
    "rec_received": {
        "name": "Case Request for Recommendation",
        "owner": "Prosecutor",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Received",
        "internal_status": "Attorney has negotiated a favorable plea offer",
        "client_status": "",
        "next_case_status": "REC_REVIEW",
    },
    "rec_reject": {
        "name": "Case Request for Recommendation",
        "owner": "Prosecutor",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Rejected",
        "internal_status": "Case cannot be handled by our attorneys",
        "client_status": "",
        "next_case_status": "REFUND",
    },
    "rec_delay": {
        "name": "Case Request for Recommendation",
        "owner": "Prosecutor",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Delayed",
        "internal_status": "The Prosecutor has delayed providing a plea offer. Please provide the information requested",
        "client_status": "",
        "next_case_status": "CLIENT_REQUEST, REC_RECEIVED, REC_REVIEW",
    },
    "rec_review": {
        "name": "Case Plea Negotiation with the Prosecutor (if applicable)",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Review",
        "internal_status": "The Attorney has initiated further negotiations with the prosecutor",
        "client_status": "",
        "next_case_status": "REC_REVIEW, REC_PROPOSAL",
    },
    "rec_proposal": {
        "name": "Case Recommendation Proposal",
        "owner": "Client",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Recommendation Proposed to Client",
        "internal_status": "Attorney has negotiated a favorable plea offer and would like you to review and accept the plea offer",
        "client_status": "",
        "next_case_status": "SIG_REQUIRED",
    },
    "sig_required": {
        "name": "Case pending Signature and Payment",
        "owner": "Client",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Recommendation pending Signature",
        "internal_status": "Client signature and payment are required to accept the Prosecutor's Plea Offer",
        "client_status": "",
        "next_case_status": "SUB",
    },
    "sub": {
        "name": "Case Recommendation and Payment Submission to Court",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Recommendation to Submit to Court",
        "internal_status": "Attorney is submitting the signed plea and payment to the court for approval",
        "client_status": "",
        "next_case_status": "SUB_REVIEW",
    },
    "sub_review": {
        "name": "Case Recommendation and Payment Review by the Court",
        "owner": "Court",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Recommendation Submission under Review by the Court",
        "internal_status": "Plea Offer is pending Court Approval",
        "client_status": "",
        "next_case_status": "APPEAR, CLOSE",
    },
    "appear": {
        "name": "Court Representation (if applicable)",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Court Appearance Required",
        "internal_status": "Court is requiring an appearance to approve the plea",
        "client_status": "",
        "next_case_status": "CLOSE",
    },
    "close": {
        "name": "Case Closure",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Close Case on Portal",
        "internal_status": "Congratulations! Case closed",
        "client_status": "",
        "next_case_status": "CLIENT_REVIEW",
    },
    "client_review": {
        "name": "Case Feedback",
        "owner": "closed",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Client Submits Review",
        "internal_status": "Please provide us with a review and feedback from your experience",
        "client_status": "",
        "next_case_status": "CLIENT_FEEDBACK",
    },
    "closed": {
        "name": "Case Closed",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "closed",
        "short_description": "Case Closed",
        "internal_status": "Case closed",
        "client_status": "",
        "next_case_status": "",
    },
}

### Function and Rules to update the status of the case

In [89]:
"""
Information received from the court	2113	=> Update the case	=> Update the case status to court_response
Reply on a request for recommendation	286	=> Update the case	=> Update the case status to rec_received
Returned for Filing 					168	=> Update the case 	=> Update the case status to court_returned_for_filing
Update the case 						565 => Update the case	=> Update the case status to case_update
"""

rules = [
    {
        # Sender shouldnt be meyerattorneyservices@gmail.com
        "regex": r"meyerattorneyservices",
        "case_status": None,
        "scope": "sender",
    },
    {
        # Sender shouldnt be Shawn-Anthony Meyer
        "regex": r"shawn-anthony meyer",
        "case_status": None,
        "scope": "sender",
    },
    {
        "regex": r"returned for filing",
        "case_status": "court_returned_for_filing",
        "scope": "subject",
    },
    {
        "regex": r"request for recommendation",
        "case_status": "rec_received",
        "scope": "subject",
    },
    {
        "regex": r"enotice",
        "case_status": "court_response",
        "scope": "subject",
    },
]

In [90]:
def get_action_from_rules(email_dict, rules):
    action = None
    for rule in rules:
        if rule["scope"] == "subject":
            match = re.search(rule["regex"], email_dict["subject"], re.IGNORECASE)
            if match:
                action = rule["case_status"]
                break
        elif rule["scope"] == "body":
            match = re.search(rule["regex"], email_dict["body"], re.IGNORECASE)
            if match:
                action = rule["case_status"]
                break
        elif rule["scope"] == "sender":
            match = re.search(rule["regex"], email_dict["sender"], re.IGNORECASE)
            if match:
                break
    return action

## Email Processing

### Action Identification for Each Email

In [91]:
emails_list = []

for email in emails:
    case_number = extract_number_from_string(gmail_connector.get_email_subject(email))
    subject = gmail_connector.get_email_subject(email)
    if case_number is None:
        case_number = extract_number_from_string(gmail_connector.get_snippet(email))
    sender = gmail_connector.render_email_sender(gmail_connector.get_sender(email))
    timestamp = gmail_connector.get_timestamp(email).strftime("%Y-%m-%d")
    email_dict = {
        "case_number": case_number,
        "subject": subject,
        "sender": sender,
        "timestamp": timestamp,
        "snippet": gmail_connector.get_snippet(email),
        "id":  email.get("id"),
    }

    action = get_action_from_rules(email_dict, rules)
    
    if action:
        email_dict["action"] = action

    emails_list += [email_dict]

In [92]:
import pandas as pd 
# Change the backend for pandas to plotly
pd.options.plotting.backend = "plotly"

emails_df = pd.DataFrame(emails_list)
emails_df["timestamp"] = pd.to_datetime(emails_df["timestamp"])

### Emails Received by Date

In [93]:
# Plot the number of emails received per day
emails_df["timestamp"].value_counts().sort_index().plot()

/Users/aennassiri/Projects/Personal/ticket-washer/venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Actions Breakdown

In [94]:
# Number of emails with actions 
emails_df["action"].value_counts().plot(kind="bar")

### Filter on 2024 and Group Emails by Case Number

In [95]:
# Focus on 2024 
emails_df = emails_df[emails_df["timestamp"].dt.year == 2024]

In [96]:
# Sort the emails by timestamp
emails_df = emails_df.sort_values(by="timestamp", ascending=False)

In [97]:
# Count cases with many emails 
emails_df.groupby("case_number").timestamp.count().sort_values(ascending=False)

case_number
72934           83
23JE-CR03544    39
24SA-CR00019    37
704283738       32
704286014       31
                ..
220572534        1
220269058        1
220236841        1
220236840        1
035361           1
Name: timestamp, Length: 991, dtype: int64

In [98]:
# Group the emails by case number and keep the last email
# Aggregate the actions first take the first and second take the unique values
cases_focus = emails_df.groupby("case_number").agg(
    {
        "timestamp": "last",
        "action": "first",
        "subject": "first",
        "sender": "first",
    }
).reset_index()

### Case Example : 23JE-CR03544

In [99]:
# Focus on 23JE-CR03544
emails_df[emails_df["case_number"] == "23JE-CR03544"]


,case_number,subject,sender,timestamp,snippet,id,action
525,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-09,ELECTRONIC NOTICE OF ENTRY (Supreme Court Rule...,18ec02dac8a853db,court_response
965,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-03,ELECTRONIC NOTICE OF ENTRY (Supreme Court Rule...,18ea5ba3cf061fc3,court_response
969,23JE-CR03544,RE: Motion to Recall Warrant - 23JE-CR03544 - ...,Kara Riffe,2024-04-03,I will consent to recalling the warrant for yo...,18ea5a8761c6f504,NaN
1036,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-03,ELECTRONIC NOTICE OF ENTRY (Supreme Court Rule...,18ea4c3135784715,court_response
1007,23JE-CR03544,Returned for Filing - 23JE-CR03544 - ST V MICH...,Missouri Courts eFiling System,2024-04-03,Your submission EF34288705 was RETURNED TO FIL...,18ea52cd13525b0d,court_returned_for_filing
1011,23JE-CR03544,Receipt of Submission - 23JE-CR03544 - ST V MI...,Missouri Courts eFiling System,2024-04-03,Your submission on 4/3/24 at 1:00 PM was RECEI...,18ea5200b6686032,NaN
1012,23JE-CR03544,Motion to Recall Warrant - 23JE-CR03544 - ST V...,Shawn-Anthony Meyer,2024-04-03,"Kara, I am reaching out regarding the warrant ...",18ea51eb353b14bd,NaN
1006,23JE-CR03544,Returned for Filing - 23JE-CR03544 - ST V MICH...,Missouri Courts eFiling System,2024-04-03,Your submission EF34288705 was RETURNED TO FIL...,18ea52cd192f3a58,court_returned_for_filing
1016,23JE-CR03544,Receipt of Submission - 23JE-CR03544 - ST V MI...,Missouri Courts eFiling System,2024-04-03,Your submission on 4/3/24 at 12:35 PM was RECE...,18ea50b642b404a2,NaN
1015,23JE-CR03544,Accepted for Filing - 23JE-CR03544 - ST V MICH...,Missouri Courts eFiling System,2024-04-03,Your submission EF34288127 was ACCEPTED FOR FI...,18ea5115aeb08756,NaN


In [100]:
cases_focus[cases_focus["case_number"] == "23JE-CR03544"]

,case_number,timestamp,action,subject,sender
639,23JE-CR03544,2024-01-31,court_response,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System


### Export the Emails and Actions to CSV

In [101]:
emails_df.to_csv("emails.csv", index=False)

In [102]:
cases_focus.to_csv("cases.csv", index=False)

### Function to Refresh the Case Number from CaseNet

In [12]:
from src.loader.leads import CaseNet
from src.models import cases as cases_model
from src.services import cases, leads
from src.services.settings import get_account


def refresh_from_casenet(case_id):
    try:
        case_id = str(case_id)
        case_details = cases.get_single_case(case_id)
        case_net_account = get_account("case_net_missouri")
        case_net = CaseNet(
            url=case_net_account.url,
            username=case_net_account.username,
            password=case_net_account.password,
        )
        if case_details is None:
            case_details = {
                "case_id": case_id,
            }
            create = True
        else:
            case_details = case_details.model_dump()
            case_details["court_code"] = case_details.get("court_id")
            create = False

        case_refreshed = case_net.refresh_case(case_details)
        print(f"Disposition : {case_refreshed['case_dispositiondetail']}")

        case_refreshed_obj = cases_model.Case.model_validate(
            case_refreshed
        )
        print(f"Disposition : {case_refreshed_obj.disposed}")

        if create is False:
            cases.update_case(case_refreshed_obj)
        else:
            cases.insert_case(case_refreshed_obj)
            leads.insert_lead_from_case(case_refreshed)
    except Exception as e:
        print(f"Error refreshing case {case_id}")

In [13]:
# Reload the module to get the latest changes

from importlib import reload
import src.loader.mycase as mycase_connector
reload(mycase_connector)

<module 'src.loader.mycase' from '/Users/aennassiri/Projects/Personal/ticket-washer/src/loader/mycase.py'>

In [14]:
mycase = mycase_connector.MyCase(url="", password="", username="")
mycase.login()

In [15]:
def get_status_from_mycase(case_id):
    mycase_details = mycase.search_case(case_id)

    if mycase_details is None:
        print(f"Case {case_id} not found on MyCase")
    else:
        mycase_id = mycase_details.get("record_id")
        case_feed = mycase.get_case_feed(mycase_id)
        closed = False
        for item in case_feed.get("data"):
            print(f"{item['created_at']} \t {item['notification_type']:20} \t{item['display_text']}")
            if "closed" in item["display_text"].lower():
                closed = True
        if closed:
            print("Case is closed on MyCase")
        return {
            "case_id": mycase_id,
            "case_feed": case_feed,
            "case_details": mycase_details,
            "closed": closed,
        }

In [17]:
# Case#230347463
info = get_status_from_mycase("230347463")

In [116]:
from time import sleep

### Process and Update the Case Status on Fubloo

In [117]:
# Iterate through case_focus and get the case status
for case_number, rows in emails_df.groupby("case_number"):
    print(f"\n\n#### Processing case {case_number}")
    row = rows.iloc[0]
    case_number = row["case_number"]
    if "fax" in row["sender"].lower() or len(case_number) <= 6 or "experian" in row["sender"].lower() or "meyerattorneyservices" in row["sender"].lower():
        continue
    # refresh_from_casenet(case_number)
    try:
        case = get_single_case(case_number)

    except Exception as e:
        print(f"Issue with case {case_number}")
        print(e)
        continue
    if case is None:
        print(f"Case {case_number} not found in the database. Trying to refresh from casenet")
        # refresh_from_casenet(case_number)
        case = get_single_case(case_number)

    if case is None:
        print(f"Case {case_number} not found in the database")
    else:
        # Update the emails attached to the case
        case_emails = rows.to_dict(orient="records")
        if case.emails is None:
            case.emails = []
        emails_list = case.emails + [
            e
            for e in case_emails
            if e.get("id") not in [i.get("id") for i in case.emails]
        ]
        # sorted_emails = sorted(
        #     emails_list, key=lambda x: x["timestamp"].tz_localize(None), reverse=True
        # )
        patch_case(case_number, {"emails": emails_list})

        # Check the case on Mycase
        mycase_details = get_status_from_mycase(case_number)

        if mycase_details is not None:
            if mycase_details["closed"]:
                case_status = "closed" 

        # Update the case status
        case_status = row["action"]
        if case.disposed:
            case_status = "client_review"

        if case.status is None and case_status is None:
            case_status = "paid"

        if case.status is not None and case_status is None:
            case_status = case.status

        if case.status != case_status and case.status != "closed" and case_status is not None and pd.notnull(case_status):
            print(f"Updating case {case_number} from {case.status} to {case_status}")
            patch_case(case_number, {"status": case_status, "flag": case_statuses[case_status]["flag"]})
        # else:
        #     print(f"Case {case_number} is already {case_status}")
        #     patch_case(
        #         case_number,
        #         {
        #             "status": case_status,
        #             "flag": case_statuses[case_status]["flag"],
        #         },
        #     )

        if case.status == "closed":
            print(f"Case {case_number} is closed")
            patch_case(case_number, {"flag": "closed"})
    sleep(1)



#### Processing case 035361


#### Processing case 068836


#### Processing case 10016939490803


#### Processing case 117766


#### Processing case 12066866071


#### Processing case 13128967900


#### Processing case 13143646300


#### Processing case 13147382288


#### Processing case 14024717817


#### Processing case 140942929
2024-04-01T23:29:33+00:00 	 updated              	updated document OFFER_and_plea_Caldwell.pdf
2024-04-01T23:22:51+00:00 	 updated              	updated task F/U client payment of invoice
2024-04-01T23:19:38+00:00 	 added                	uploaded a new document OFFER_and_plea_Caldwell.pdf
2024-02-09T17:48:29+00:00 	 updated              	updated task F/U client payment of invoice
2024-02-09T17:46:12+00:00 	 added                	uploaded a new document Cadwell_need_insurance_.pdf
2024-01-29T20:18:05+00:00 	 added                	uploaded a new document Insurance Card
2024-01-26T20:39:33+00:00 	 added                	added task F/U client payment of invoice

In [110]:

pd.notna(case_status)

False

In [91]:
case_number

'10016939490803'

In [92]:
case_emails

[{'case_number': '10016939490803',
  'subject': 'Your monthly account statement is here, Shawn',
  'sender': 'Experian',
  'timestamp': Timestamp('2024-04-08 00:00:00'),
  'snippet': 'Review your account statement Experian Logo Membership ID# 10016939490803 Sign In Check out your monthly account statement 2 unread notifications 2 new in the past 30 days Your credit rating is fair',
  'id': '18ebe51889b73112',
  'action': nan}]

In [61]:
emails_by_sender = emails_df.groupby("sender").id.count().sort_values(ascending=False)

In [63]:
emails_by_sender.to_csv("emails_by_sender.csv")

In [22]:
import json
import os

from langchain.callbacks import get_openai_callback
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

In [112]:
def get_email_info_structured(text: str) -> str:
    """
    This function would return the result in JSON structure only.
    We don't have to write a custom parse to convert string to JSON LangChain already provides with the a parser
    """
    response_schemas = [
        ResponseSchema(name="Summary", description="Summary of the email"),
        ResponseSchema(
            name="CaseNumber",
            description="Case Number extracted from the email",
        ),
        # Sentiment Analysis
        ResponseSchema(
            name="Sentiment",
            description="Sentiment of the email",
        ),
        # Next Steps 
        ResponseSchema(
            name="NextCaseStatus",
            description="Next Case Status",
        ),
        # Next Step Owner 
        ResponseSchema(
            name="NextStepOwner",
            description="Next Step Owner",
        )
    ]
    output_parser = StructuredOutputParser.from_response_schemas(
        response_schemas
    )

    format_instructions = output_parser.get_format_instructions()
    prompt = PromptTemplate(
        template=f"""Get the summary of the email and extract the case number from the email and provide the sentiment of the email and the next case status and the next step owner. The current process is as follows: {case_statuses}""",
        input_variables=["text"],
        partial_variables={"format_instructions": format_instructions},
    )

    chain = prompt | llm | output_parser
    with get_openai_callback() as cb:
        response = chain.invoke({"filed": text})
        print(f"Total Cost (USD): ${format(cb.total_cost, '.4f')}")

    return response

In [113]:
final_body, text_body = gmail_connector.get_email_html_body(email)

In [116]:
text_body

prompt = f"""Get the summary of the email and extract the case number from the email and provide the sentiment of the email and the next case status and the next step owner. #### The email :{text_body} #### The current process is as follows: {case_statuses}"""

In [108]:
final_body, text_body = gmail_connector.get_email_html_body(email)

In [109]:
text_body

"      \r I can amend to Defective Equipment, $200 total. I'll send you a copy of the amended ticket for filing with payment. Joe Cambiano   From: meyerattorneyservices@gmail.com <meyerattorneyservices@gmail.com> Sent: Tuesday, April 2, 2024 12:45 PM To: Joe Cambiano <joe@kcinjurylaw.com> Subject: Request for Recommendation - BRYCEN DON THORNTON - 190873126   Hello, \r I hope this email finds you well! \r I am reaching out on behalf of my client, BRYCEN DON THORNTON, who has sought my representation in a criminal matter presently overseen by your office. The details pertaining to the case are as follows: \r Case #: 190873126\r Charge: Fail To Stop At Stop Sign At Stop Line/Before Crosswalk/Point Nearest Intersection \r I am reaching out to request your recommendation regarding this matter. I have attached a copy of the citation(s) for your reference. Should you require any further information or documentation to facilitate your recommendation, please do not hesitate to let\r  me know. 

In [117]:
########################
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [121]:
system_message = f"You are an an attorney that work on traffic tickets. We received a response from the prosecutor on a case. I received from the prosecutor {gmail_connector.render_email_sender(gmail_connector.get_sender(email))} regarding a traffic ticket"

system_message_prompt = SystemMessagePromptTemplate.from_template(
    system_message
)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.run(prompt)
print(result)

Summary of the email:

The prosecutor, Joe Cambiano, responded to an attorney's request for a recommendation on behalf of their client, Brycen Don Thornton. The case number is 190873126, and the charge is "Fail To Stop At Stop Sign At Stop Line/Before Crosswalk/Point Nearest Intersection." The prosecutor suggested amending the charge to Defective Equipment with a total fine of $200. They will send a copy of the amended ticket for filing along with the payment.

Case Number: 190873126

Sentiment of the email: The email appears to be a response providing a plea offer, so the sentiment is likely neutral or slightly positive.

Next Case Status: The next case status is not mentioned in the email.

Next Step Owner: The next step owner is not mentioned in the email.
